In [1]:
import wandb
import torch
from model.train import train
from utils.configs import *
from data.CommentaryDataloader import get_commentary_dataloader
from data.CommentaryDataset import CommentaryDataset
from model.predict import Predictor

In [2]:
def get_configs_from_wandb_config(wandb_config: WandbConfig, is_real_wandb: bool):
    shared_config: SharedConfig = {
        'context_length': wandb_config['context_length'],
        'sentencepiece_path': f"./artifacts/sp{wandb_config['sp_vocab']}.model",
        'bos_id': 0, # will be initialized later
        'eos_id': 1, # will be initialized later
        'pad_id': 2, # will be initialized later
        'vocab_size': 0 # will be initialized later
    }
    
    train_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'train',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': wandb_config['samples_per_train_epoch'],
        'dl_num_workers': 2,
    }
    
    valid_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': True,
        'dl_shuffle': True,
        'dl_samples': None,
        'dl_num_workers': 1,
    }
    
    test_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': None,
        'dl_num_workers': 0,
    }
    
    model_config: ModelConfig = {
        'board_embedding_size': wandb_config['board_embedding_size'],
        'text_embedding_size': wandb_config['text_embedding_size'],
        'conv_modules_count': wandb_config['conv_modules_count'],
        'transformer_blocks': wandb_config['transformer_blocks'],
        'board_intermediary_channels': wandb_config['board_intermediary_channels'],
        'board_in_channels': CommentaryDataset.get_board_channels(train_data_config),
        'board_height': 8,
        'board_width': 8,
        'board_depth': wandb_config['board_embedding_size'],
        'ff_inner_channels': wandb_config['ff_inner_channels'],
        'num_heads': wandb_config['num_heads']
    }
    
    if wandb_config['optimizer'] not in ['adam', 'sgd']:
        raise ValueError('Optimizer value invalid')
    
    train_config: TrainConfig = {
        'lr': wandb_config['lr'],
        'with_wandb': is_real_wandb,
        'num_epochs': wandb_config['num_epochs'],
        'predict_sentences': wandb_config['predict_sentences'],
        'optimizer': Optimizers.ADAM if wandb_config['optimizer'] == 'adam' else Optimizers.SGD
    }
    
    return {
        'shared_config': shared_config,
        'train_config': train_config,
        'model_config': model_config,
        'test_data_config': test_data_config,
        'train_data_config': train_data_config,
        'valid_data_config': valid_data_config
    }     

In [6]:
local_wandb_config: WandbConfig = {
    'text_embedding_size': 64,
    'conv_modules_count': 1,
    'transformer_blocks': 1,
    'board_intermediary_channels': 64,
    'board_embedding_size': 64,
    'ff_inner_channels': 64,
    'num_heads': 4,
    'lr': 0.01,
    'optimizer': 'adam',
    'num_epochs': 10,
    'context_length': 64,
    'sp_vocab': 800,
    'batch_size': 64,
    'past_boards': 1,
    'stride_big_sequences': 64,
    'samples_per_train_epoch': 20,
    'predict_sentences': 4
}

In [7]:
configs = get_configs_from_wandb_config(local_wandb_config, False)

In [ ]:
train(
    model_config=configs['model_config'],
    train_config=configs['train_config'],
    train_dl=get_commentary_dataloader(configs['train_data_config'], configs['shared_config']),
    val_dl=get_commentary_dataloader(configs['valid_data_config'], configs['shared_config']),
    test_ds=CommentaryDataset(configs['test_data_config'], configs['shared_config']),
    predictor=Predictor(configs['shared_config'])
)